# Bi-LSTM pour l'étiquetage morpho-syntaxique
Application du tutoriel https://nlpforhackers.io/lstm-pos-tagger-keras/

In [1]:
import nltk
nltk.download("treebank")
 
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[nltk_data] Downloading package treebank to /Users/mac/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


In [2]:
import numpy as np
 
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
# Let's see how a sequence looks
 
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [3]:
from sklearn.model_selection import train_test_split
 
 
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [4]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [5]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[8515, 2721, 4067, 3457, 10094, 4067, 2688, 10152, 1335, 1667, 7608, 5032, 8330, 1419, 1485, 5791, 6987, 853, 3274, 390, 4067, 8430, 5223, 8931, 4067, 6987, 1253, 3113, 2576, 7128, 9462, 7472, 2721, 4067, 6987, 516, 2217, 7284]
[1, 1, 1182, 4375, 2152, 8648, 1, 1, 7727, 390, 7758, 9120, 4067, 7458, 9038, 5032, 1, 1279, 7284]
[6, 6, 10, 1, 23, 10, 14, 14, 26, 12, 7, 31, 12, 1, 7, 6, 14, 30, 1, 6, 10, 1, 5, 40, 10, 14, 6, 41, 25, 1, 6, 30, 6, 10, 14, 25, 30, 18]
[14, 14, 43, 14, 27, 10, 25, 14, 30, 44, 7, 27, 10, 30, 7, 31, 12, 1, 18]


In [6]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

271


In [7]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[ 8515  2721  4067  3457 10094  4067  2688 10152  1335  1667  7608  5032
  8330  1419  1485  5791  6987   853  3274   390  4067  8430  5223  8931
  4067  6987  1253  3113  2576  7128  9462  7472  2721  4067  6987   516
  2217  7284     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [10]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 271, 128)          1302400   
                                                                 
 bidirectional_1 (Bidirectio  (None, 271, 512)         788480    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 271, 47)          24111     
 tributed)                                                       
                                                                 
 activation_1 (Activation)   (None, 271, 47)           0         
                                                                 
Total params: 2,114,991
Trainable params: 2,114,991
Non-trainable params: 0
_________________________________________________________________


In [11]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [12]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [13]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
20/20 [==============================] - 128s 6s/step - loss: 1.2140 - accuracy: 0.8578 - val_loss: 0.3867 - val_accuracy: 0.9049
Epoch 2/40
20/20 [==============================] - 123s 6s/step - loss: 0.3381 - accuracy: 0.9053 - val_loss: 0.3237 - val_accuracy: 0.9050
Epoch 3/40
20/20 [==============================] - 132s 7s/step - loss: 0.3171 - accuracy: 0.9083 - val_loss: 0.3137 - val_accuracy: 0.9156
Epoch 4/40
20/20 [==============================] - 121s 6s/step - loss: 0.3078 - accuracy: 0.9161 - val_loss: 0.3051 - val_accuracy: 0.9161
Epoch 5/40
20/20 [==============================] - 119s 6s/step - loss: 0.2992 - accuracy: 0.9168 - val_loss: 0.2980 - val_accuracy: 0.9164
Epoch 6/40
20/20 [==============================] - 124s 6s/step - loss: 0.2909 - accuracy: 0.9174 - val_loss: 0.2920 - val_accuracy: 0.9177
Epoch 7/40
20/20 [==============================] - 131s 7s/step - loss: 0.2833 - accuracy: 0.9193 - val_loss: 0.2854 - val_accuracy: 0.9192
Epoch 8/40
20

In [14]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

25/25 [==============================] - 22s 891ms/step - loss: 0.0361 - accuracy: 0.9911
accuracy: 99.10553097724915


In [16]:
test_samples = [
    "running is very important for me .".split(),
    "I was running every day for a month .".split()
]
print(test_samples)

[['running', 'is', 'very', 'important', 'for', 'me', '.'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


In [17]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[7553 7224 5906  459 4987  689 7284    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [18]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[2.18290277e-03 1.93107054e-02 2.30315495e-02 ... 2.40547908e-03
   7.02051399e-03 1.68226357e-03]
  [9.15750570e-06 6.66147462e-05 2.70896533e-04 ... 2.05056864e-07
   2.27354365e-04 5.29999261e-06]
  [1.71216088e-04 2.22155518e-06 3.99776618e-04 ... 7.46191836e-06
   4.49165556e-04 1.65691727e-05]
  ...
  [9.99987960e-01 2.31973889e-08 3.66844603e-07 ... 5.10959830e-10
   1.69985265e-06 2.70491007e-08]
  [9.99980330e-01 1.51965160e-08 5.13104737e-07 ... 1.59485392e-09
   3.21016682e-06 4.85386771e-08]
  [9.99967813e-01 1.15583942e-08 7.00975249e-07 ... 4.24078106e-09
   5.57118392e-06 8.04772995e-08]]

 [[3.40510014e-04 7.23261561e-04 1.00592431e-03 ... 2.07038550e-03
   5.51617821e-04 2.38451321e-04]
  [1.32522528e-05 1.19818076e-04 3.93283699e-04 ... 1.53770301e-07
   1.35581198e-04 9.28573627e-06]
  [8.75950092e-04 1.41816388e-04 2.30100248e-02 ... 1.94781198e-04
   9.16466210e-03 5.44423005e-04]
  ...
  [9.99987960e-01 2.31973889e-08 3.66844603e-07 ... 5.10959830e-10
   1.69985

In [19]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [20]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['VBG', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

In [21]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [22]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 271, 128)          1290624   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_2 (Activation)    (None, 271, 47)           0         
Total params: 2,103,215
Trainable params: 2,103,215
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

C:\Users\Amal Zouaq\Anaconda3\envs\keras_env\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 268s 107ms/step - loss: 1.1602 - accuracy: 0.9021 - ignore_accuracy: 0.0291 - val_loss: 0.3754 - val_accuracy: 0.9114 - val_ignore_accuracy: 0.1228
Epoch 2/40
2504/2504 [==============================] - 266s 106ms/step - loss: 0.3380 - accuracy: 0.9084 - ignore_accuracy: 0.0725 - val_loss: 0.3252 - val_accuracy: 0.9072 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
2504/2504 [==============================] - 265s 106ms/step - loss: 0.3154 - accuracy: 0.9097 - ignore_accuracy: 0.0999 - val_loss: 0.3039 - val_accuracy: 0.9191 - val_ignore_accuracy: 0.1367
Epoch 4/40
2504/2504 [==============================] - 267s 107ms/step - loss: 0.3047 - accuracy: 0.9162 - ignore_accuracy: 0.1326 - val_loss: 0.3083 - val_accuracy: 0.9190 - val_ignore_accuracy: 0.1374
Epoch 5/40
2504/2504 [==============================] - 266s 106ms/step - loss: 0.2959 - accuracy: 0.9170 - ignore_accuracy: 0.1356 -

In [24]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['NNS', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

In [26]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))


783/783 [==============================] - 52s 66ms/step
ignore_accuracy: 90.68601727485657


In [27]:
print(f"{model.metrics_names[2]}: {scores[2] * 100}")

ignore_accuracy: 90.68601727485657
